 # Implementing a Simple RAG Pipeline


install all the necessary libraries

In [ ]:
!pip install langchain_community



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.5 MB/s eta 0:00:00


import the necessary libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

Scrape the website content. We can also make a custom Dataset or use pdf data.

In [ ]:

# Web scraping function
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    return [p.get_text() for p in paragraphs]

# Clean scraped content
def clean_content(content_list):
    unwanted_items = {'Sign up', 'Sign in', 'Follow', '--', '15', 'Listen'}
    cleaned = [text for text in content_list if text and text not in unwanted_items]
    return cleaned

# Scrape and clean website content
url = 'https://medium.com/@akriti.upadhyay/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7'
web_content = scrape_website(url)
cleaned_content = clean_content(web_content)

# Convert cleaned content to Documents
documents = [Document(page_content=text) for text in cleaned_content]


splitting the documnets and creating the embeddings

In [ ]:

# Split the documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


using a gpt2-large model for better generation

In [ ]:

# Set up a more capable language model
model_name = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Creating the pipeline with max tokens 50

In [ ]:
# Create a text-generation pipeline with adjusted parameters
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True
)

# Create a HuggingFacePipeline language model
local_llm = HuggingFacePipeline(pipeline=pipe)


Creating a prompt template to get better output for our query

In [ ]:

# Set up the retrieval-based QA system with a simplified prompt template
template = """Context: {context}

Question: {question}

Answer the question concisely in one sentence based only on the given context:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
)


In [ ]:

# Simplified RAG Pipeline function
def rag_pipeline(query):
    response = qa.run(query)
    # Simple post-processing: take the first sentence
    first_sentence = response.split('.')[0] + '.'
    return first_sentence


Generating responses

In [ ]:

# Example usage
query = "What embedding is captured?"
result = rag_pipeline(query)
print(f"Query: {query}")
print(f"RAG Response: {result}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What embedding is captured?
RAG Response: Context: Embeddings capture the semantic meaning of the text which allows you to quickly and efficiently find other pieces of text which are similar.
